# Model Predictive Control Project

The aim of this project is to design a model predictive controller(MPC) to drive a vehicle in the simulator. MPC controller is an optimal controller and it is different from a PID controller. A PID controller calculates the acutator commands based on the propotional, integral and derivative operations of the error signal. Whereas a MPC controller predicts the system outputs for a finite number of steps in future and calculates the acutator commands by minimizing a cost function (function of error, usage of actuator etc.). MPC algorithm takes into account the actuator contrains and time delays.With mpc it is possible to acheive higher system performance solve more complex problems. The disadvantages of MPC are high computation intensive and it is harder to define the stability of the system with MPC.

The following picture shows the overview of the MPC.

![MPC](./MPC.png)

The following points shows the steps taken to design the MPC controller:
- Estimation or Measurement of system state
- Definition of the Reference and calculation of error
- MPC calculates the actuator commands($u_0$) for N steps in future to minimize the cost function 
$J = \sum_{i=0}^N f(x_{t+i},r_{t+i},u_{t+i})$.
- Actuator command $u_0$ is applied for the current cycle and the whole process is repeated again for next cycle.

![MPC](./MPC-vis.jpg)



The designing of MPC for this project involves designing the following

- System Model
- Calculation of Cross Track Error and Heading Error
- Sample Time(dt) and No. of steps in prediction horizon(N)
- Cost Function
- Adapting  the system for actuator delay


## Vehicle Model 

There are two ways to describe the lateral dynamics of a car such as kinematic models, dynamic models. The kinematic models consider only the geometry of the vehicle motion and the car to decribe the behavior. It is simple, less computation intensive and has lesser accuracy compared to dynamic model. The dynamic models considers not only the geometry but also the forces acting on the vehicle to describe the vehicle motion. In this project only a non-linear kinematic model is used to describe the motion of the vehicle. Kinematic model is used because it is less computation intensive and it is has sufficient accuracy to describe the motion of a car in the simulator.The bicycle motion model discussed in the lecture is used in the project. The following picture shows the bicycle model of a car.

![MPC](./SingleTrackModel.png)

The following quatities are considered as state variables:
- Position of the car in X-Axis ($x_t$)
- Position of the car in Y-Axis ($y_t$)
- Orientation of the car along Z-Axis ($\psi_t$)
- Magnitude of vehicle velocity ($v_t$)
- Cross Track Error($cte_t$)- minimum distance between actual car position and reference trajectory
- Heading Error($e\psi_t$) - difference between refenrece heading and the actual heading

In this model the error values $cte_t$ and $e\psi_t$ are also considered as state variable because the MPC uses the error values to calculate the cost and minimizes them.

The following equations describe the motion of the car in 2D plane. It has to be noted that the side slip angle i.e. the angle between the velocity vector and the orientation of the car is zero.

$ x_{t+1}    = x_t + v_t cos(\psi_t)dt $

$ y_{t+1}    = y_t + v_t sin(\psi_t)dt $

$ \psi_{t+1} = \psi_t + \frac{v_t}{L_f}{\delta_t}dt$

$ v_{t+1} = v_t + a_tk_adt$

$ cte_{t+1} = f(x_t) - y_t + v_tsin(e\psi_t)dt$

$ e\psi_t+1 = {\psi}des_t + \frac{v_t}{L_f}{\delta_t}dt$

where:
- $dt$ is the sample time
- $\delta_t$ is the steering angle
- $L_f$ is the distance between front wheel and car's center of gravity which is given in the lecture
- $a_t$ is the throttle position
- $k_a$ is the gain between the throttle position and the acceleration. This value is chosen based on the experiment in the simulator

